# Exam - Feb, 24th 2022

## Part 1 - MCQ

1. d - K-Means is a clusterig algorithm
2. c - 5 blocks

## Part 2 - programming

### Ex 1


In [ ]:
# With DataFrames

input_catal = './ItemsCatalog.txt'
input_sales = './Sales.txt'

out_1 = 'outputFolderEx1'

# Sales DF
in_sales_DF = spark.read.load(input_sales, format='csv', header=False)\
                    .toDF('timestamp', 'userID', 'itemID', 'salePrice')

# Catalog DF - only keep useful columns
in_cat_DF = spark.read.load(input_catal, format='csv', header=False)\
                    .toDF('itemID', 'name', 'category', 'firstTimeIn', 'suggPrice')\
                    .select('itemID', 'suggPrice')

# UDF for getting the year
spark.udf.register('getYear', lambda ts: int(ts.split('/')[0]))

# Keep sales from 2021 only
sales_2021_DF = in_sales_DF.filter('getYear(timestamp) == 2021')

# Join 2021 sales with catalog to get suggested price
both_prices_DF = sales_2021_DF.join(in_cat_DF, sales_2021_DF.itemID == in_cat_DF.itemID)

both_prices_DF.createOrReplaceTempView('my_table')

# Directly obtain the DF containing the ratio between the number of sales
# at more than the suggeted price and the total sales
out_DF = spark.sql("""
            SELECT itemID, sum(CAST(salePrice > suggPrice AS int))/count(*) AS fract_more
            FROM my_table
            GROUP BY itemID
""").filter('fract_more > 0.8')

out_DF.select('itemID').write.csv(out_1, header=False)


### Ex 2

#### 2.1

In [ ]:
input_catal = './ItemsCatalog.txt'
input_sales = './Sales.txt'
input_cust = './Customers.txt'

out_2_1 = 'outFolderEx2_1'
out_2_2 = 'outFolderEx2_2'

########################################

in_sales_DF = spark.read.load(input_sales, format='csv', header=False)\
                    .toDF('timetamp', 'userID', 'itemID', 'salePrice')
in_sales_DF.createOrReplaceTempView('sales_table')

spark.udf.register('getYear', lambda ts: int(ts.split('/')[0]))

# In the output, users such that:
# - Sales from 2020
# - Number of different items > 100
# - Total money spent > 1000
out_2_1_DF = spark.sql("""
            SELECT userID, count(distinct(itemID)) AS n_items, sum(salePrice) AS tot_spent
            FROM sales_table
            WHERE getYear(timestamp) == 2020
            GROUP BY userID
            HAVING count(distinct(itemID)) > 100 AND sum(salePrice) > 1000
""").select('userID')

out_2_1_DF.write.csv(out_2_1, header=False)


#### 2.2

In [ ]:
out_2_1_DF.createOrReplaceTempView('good_2020')

spark.udf.register('getMonth', lambda ts: int(ts.split('/')[1]))

# Start by selecting only sales of 2021 and joining 
# the filtered sales with the good customers of 2020:

all_2021 = spark.sql("""
            SELECT sal.userID, getMonth(sal.timestamp) AS month
            FROM sales_table AS sal, good_2020
            WHERE sal.userID == good_2020.userID
""")

all_2021.createOrReplaceTempView('t1')

########## maybe can transform these two (^ and v) into a single query (not sure)

# Select couples (userID, month) having less than 3 sales (requirement for being classified as good customer)
good_2021 = spark.sql("""
            SELECT userID, month, count(*) AS n_sales_mo
            FROM t1
            GROUP BY userID, month
            HAVING count(*) < 3
""")

customers_DF = spark.read.load(input_cust, format='csv', header=False)\
                    .toDF('userID', 'name', 'surname', 'birth')\
                    .select('userID', 'surname')

# Isolate these customers who have all 12 months left in the previous DF
good_customers_2021 = good_2021.groupBy('userID').agg({'*': 'count'})\
                            .withColumnRenamed('count(1)', 'ok_months')\
                            .filter('ok_months == 12').select('userID')\

# Join on the customers DF (with just ID and surname) to get surname
good_surname_2021 = good_customers_2021.join(customer_DF, good_customers_2021.userID == customer_DF.userID)

good_surname_2021.write.csv(out_2_2, header=False)

In [ ]:
############## IT IS POSSIBLE TO MERGE THE 2 QUERIES

good_2021 = spark.sql("""
            SELECT sal.userID, getMonth(sal.timestamp) AS month, count(*) AS n_sales_mo
            FROM sales_table AS sal, good_2020
            WHERE sal.userID == good_2020.userID
            GROUP BY sal.userID, month
            HAVING count(*) < 3
""")